In [8]:
import os
import numpy as np
import torch
import tqdm
import yaml
from attrdict import AttrDict
import torch.nn.functional as F
import matplotlib.pyplot as plt
from collections import defaultdict
import json

from modules.mono.depth_net_pl import depth_net_pl
from modules.semantic.semantic_net_pl import semantic_net_pl
from data_utils.data_utils import GridSeqDataset
from data_utils.prob_vol_data_utils import ProbVolDataset
from utils.localization_utils import (
    get_ray_from_depth, get_ray_from_semantics, localize, finalize_localization
)
from utils.data_loader_helper import load_scene_data
from scipy.ndimage import gaussian_filter1d
import seaborn as sns
import pickle
from PIL import Image


# Hard-coded configuration
config = AttrDict({
    'original_dataset_dir': '/datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/data/structured3d_perspective_full/Structured3D',
    'dataset_dir': '/datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/data/test_data_set_full/structured3d_perspective_full',
    'desdf_path': '/datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/data/test_data_set_full/desdf',
    'log_dir_depth': '/datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/modules/Final_wights/depth/final_depth_model_checkpoint.ckpt',
    'log_dir_semantic': '/datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/modules/Final_wights/semantic/final_semantic_model_checkpoint.ckpt',
    'split_file': '/datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/data/test_data_set_full/structured3d_perspective_full/split.yaml',
    'prob_vol_path': '/datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/data/test_data_set_full/prob_vols',
    'results_dir': '/datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/results/full/GT',
    'combined_prob_vols_net_path': '/datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/logs/combined/expected_pose_pred/combined_prob_vols_net_type-expected_pose_pred_net_size-6k_dataset_size-medium_epochs-99/combined_net-epoch=58-loss-valid=7.16.ckpt',
    'combined_net_size': '6k',
    'L': 0,
    'D': 128,
    'd_min': 0.1,
    'd_max': 15.0,
    'd_hyp': -0.2,
    'F_W': 0.59587643422,
    'V': 7,
    'num_classes': 4,
    'prediction_type': 'combined',
    'use_ground_truth_depth': False,
    'use_ground_truth_semantic': False,
    'use_saved_prob_vol': False,
    'num_of_scenes': -1,
    'pad_to_max': True,
    'max_h': 1760,
    'max_w': 1760,
    'weight_combinations': [
        [1.0, 0.0],
        [0.5, 0.5]
    ]
})


def save_data(data, path):
    """Saves data to a specified path using pickle."""
    with open(path, 'wb') as f:
        pickle.dump(data, f)


def load_data(path):
    """Loads data from a specified path using pickle."""
    with open(path, 'rb') as f:
        return pickle.load(f)


def plot_acc_vs_count(count_dict_for_all_weights, count_name, save_path, data_type):
    """
    Plots accuracy vs a given metric (e.g., binned apartment size) for different weight combinations.
    Also includes the absolute gap between '1.0_0.0' and '0.5_0.5' accuracy as a third line.
    """
    sns.set(style="whitegrid")
    plt.figure(figsize=(12, 8))
    
    # Ensure required keys exist
    if '1.0_0.0' not in count_dict_for_all_weights or '0.5_0.5' not in count_dict_for_all_weights:
        raise ValueError("Both '1.0_0.0' and '0.5_0.5' keys must exist in count_dict_for_all_weights")

    # Plot mean accuracy for each weight combination
    for weight_key in count_dict_for_all_weights.keys():
        sorted_counts = sorted(count_dict_for_all_weights[weight_key].keys())
        mean_acc = [np.mean(count_dict_for_all_weights[weight_key][c]) for c in sorted_counts]
        smoothed_mean = gaussian_filter1d(mean_acc, sigma=1)  # Optionally apply smoothing
        
        sns.lineplot(x=sorted_counts, y=smoothed_mean, label=f'{weight_key}', linewidth=2)
        sns.lineplot(x=sorted_counts, y=mean_acc, linewidth=1, alpha=0.4)
    
    # Compute the absolute gap between '1.0_0.0' and '0.5_0.5'
    sorted_counts_gap = sorted(set(count_dict_for_all_weights['1.0_0.0'].keys()) & set(count_dict_for_all_weights['0.5_0.5'].keys()))
    gap_mean = []
    for count in sorted_counts_gap:
        depth_only_acc = np.mean(count_dict_for_all_weights['1.0_0.0'][count])
        combined_acc = np.mean(count_dict_for_all_weights['0.5_0.5'][count])
        gap = abs(depth_only_acc - combined_acc)
        gap_mean.append(gap)

    # Plot the gap as a third line
    if len(gap_mean) > 0:
        sns.lineplot(x=sorted_counts_gap, y=gap_mean, label='|Depth Only - 0.5,0.5|', linewidth=2, linestyle='--', color='red')

        # Add annotations for the gap values
        for x, y in zip(sorted_counts_gap, gap_mean):
            plt.text(x, y, f"{y:.2f}", color='red', fontsize=9, ha='center')

    # Finalize the plot
    plt.xlabel(f'{count_name}')
    plt.ylabel('Mean Accuracy (m)')
    plt.legend(title="Weight Combination and Gap")
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, f'{data_type}_acc_vs_{count_name.replace(" ", "_")}_with_gap.png'))
    plt.close()


def load_apartment_size(scene_dir):
    """
    Calculate the apartment size based on the dimensions of the floorplan_semantic.png.
    Assuming 100 pixels = 1 meter.
    """
    floorplan_path = os.path.join(scene_dir, "floorplan_semantic.png")
    if not os.path.exists(floorplan_path):
        return 0.0

    with Image.open(floorplan_path) as img:
        width, height = img.size  # dimensions in pixels

    # Convert to meters
    width_m = width / 100.0
    height_m = height / 100.0
    area_m2 = width_m * height_m
    return area_m2


def evaluate_combined_model(
    depth_net, semantic_net, desdfs, semantics, test_set, gt_poses, maps,
    device, results_type_dir, valid_scene_names, config
):
    save_path = '/datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/results/final_results/visualtizations/size_vs_acc'
    os.makedirs(save_path, exist_ok=True)
    counts_file = os.path.join(save_path, 'count_dict_for_all_weights_size_only.pkl')
    
    # Precompute apartment sizes per scene
    scene_to_size = {}
    apt_sizes = []
    for scene_name in valid_scene_names:
        scene_dir = os.path.join(config.dataset_dir, scene_name)
        apt_size = load_apartment_size(scene_dir)
        scene_to_size[scene_name] = apt_size
        apt_sizes.append(apt_size)

    # Define bins for apartment sizes
    # For example, create 10 bins between the min and max apartment size
    if len(apt_sizes) > 0:
        min_size = min(apt_sizes)
        max_size = max(apt_sizes)
        num_bins = 10 
        bin_edges = np.linspace(min_size, max_size, num_bins+1)
    else:
        # Fallback if no scenes
        bin_edges = np.array([0, 1])

    # Function to get bin index for an apartment size
    def get_bin_idx(x):
        # digitize returns the bin index such that bin_edges[bin_idx-1] <= x < bin_edges[bin_idx]
        # We'll use bin_idx - 1 to have 0-based indexing
        return np.digitize(x, bin_edges) - 1

    # Print the bin ranges
    print("\nApartment size bins (m²):")
    for i in range(len(bin_edges)-1):
        print(f"  Bin {i}: [{bin_edges[i]:.2f}, {bin_edges[i+1]:.2f})")

    if os.path.exists(counts_file) and False:
        print(f"Loading existing count data from {counts_file}")
        data = load_data(counts_file)
        size_acc = data['size_acc']
        gt_size_acc = data['gt_size_acc']
    else:
        print(f"No existing data found at {counts_file}, running calculations.")
        wc = config.weight_combinations
        size_acc = {f"{dw}_{sw}": defaultdict(list) for dw, sw in wc}
        gt_size_acc = {f"{dw}_{sw}": defaultdict(list) for dw, sw in wc}

        for data_idx in tqdm.tqdm(range(len(test_set))):
            data = test_set[data_idx]
            scene_idx = np.sum(data_idx >= np.array(test_set.scene_start_idx)) - 1
            scene = test_set.scene_names[scene_idx]
            if 'floor' not in scene:
                scene = f"scene_{int(scene.split('_')[1])}"

            if scene not in valid_scene_names:
                continue

            apartment_size = scene_to_size[scene]
            bin_idx = get_bin_idx(apartment_size)  # Get the bin index for the apartment size

            idx_within_scene = data_idx - test_set.scene_start_idx[scene_idx]
            desdf, semantic = desdfs[scene], semantics[scene]
            ref_pose_map = gt_poses[scene][idx_within_scene * (config.L + 1) + config.L]

            # Predicted depth and semantics
            ref_img_t = torch.tensor(data["ref_img"], device=device).unsqueeze(0)
            pred_depths = depth_net.encoder(ref_img_t, None)[0].squeeze(0).cpu().detach().numpy()
            pred_rays_depth = get_ray_from_depth(pred_depths, V=config.V, F_W=config.F_W)

            _, _, prob = semantic_net.encoder(ref_img_t, None)
            sampled_indices_np = torch.multinomial(prob.squeeze(0), 1, replacement=True).squeeze(1).cpu().detach().numpy()
            pred_rays_semantic = get_ray_from_semantics(sampled_indices_np, V=config.V, F_W=config.F_W)

            prob_vol_pred_depth = localize(torch.tensor(desdf["desdf"]), torch.tensor(pred_rays_depth), return_np=False)[0]
            prob_vol_pred_semantic = localize(torch.tensor(semantic["desdf"]), torch.tensor(pred_rays_semantic), return_np=False)[0]

            # Ground truth depth and semantics
            gt_pred_depths = data["ref_depth"]
            gt_rays_depth = get_ray_from_depth(gt_pred_depths, V=config.V, F_W=config.F_W)
            gt_sampled_indices_np = data["ref_semantics"]
            gt_rays_semantic = get_ray_from_semantics(gt_sampled_indices_np, V=config.V, F_W=config.F_W)

            gt_prob_vol_pred_depth = localize(torch.tensor(desdf["desdf"]), torch.tensor(gt_rays_depth), return_np=False)[0]
            gt_prob_vol_pred_semantic = localize(torch.tensor(semantic["desdf"]), torch.tensor(gt_rays_semantic), return_np=False)[0]

            # Align shapes if needed
            def align_shapes(a, b):
                min_shape = [min(d1, d2) for d1, d2 in zip(a.shape, b.shape)]
                slices = tuple(slice(0, ms) for ms in min_shape)
                return a[slices], b[slices]

            d_sliced, s_sliced = align_shapes(prob_vol_pred_depth, prob_vol_pred_semantic)
            d_gt_sliced, s_gt_sliced = align_shapes(gt_prob_vol_pred_depth, gt_prob_vol_pred_semantic)

            for dw, sw in wc:
                wkey = f"{dw}_{sw}"
                combined_prob = dw * d_sliced + sw * s_sliced
                _, pdp, _, pose_pred = finalize_localization(combined_prob)
                pose_pred = torch.tensor(pose_pred, device=device, dtype=torch.float32)
                pose_pred[:2] = pose_pred[:2] / 10
                acc = torch.norm(pose_pred[:2] - torch.tensor(ref_pose_map[:2], device=device), 2).item()
                size_acc[wkey][bin_idx].append(acc)

                gt_combined_prob = dw * d_gt_sliced + sw * s_gt_sliced
                _, gt_pdp, _, gt_pose_pred = finalize_localization(gt_combined_prob)
                gt_pose_pred = torch.tensor(gt_pose_pred, device=device, dtype=torch.float32)
                gt_pose_pred[:2] = gt_pose_pred[:2] / 10
                gt_acc = torch.norm(gt_pose_pred[:2] - torch.tensor(ref_pose_map[:2], device=device), 2).item()
                gt_size_acc[wkey][bin_idx].append(gt_acc)

        data_to_save = {
            'size_acc': size_acc,
            'gt_size_acc': gt_size_acc,
            'bin_edges': bin_edges
        }
        save_data(data_to_save, counts_file)

    # Use bin indices for plotting, label them by bin midpoint for readability
    # We'll create a new dictionary with bin midpoints as keys
    def midpoint(a, b):
        return (a + b) / 2.0

    bin_midpoints = [midpoint(bin_edges[i], bin_edges[i+1]) for i in range(len(bin_edges)-1)]

    def remap_keys(original_dict):
        new_dict = {}
        for wkey, bin_dict in original_dict.items():
            new_bin_dict = {}
            for bin_idx, values in bin_dict.items():
                if bin_idx < len(bin_midpoints):
                    new_bin_dict[bin_midpoints[bin_idx]] = values
            new_dict[wkey] = new_bin_dict
        return new_dict

    size_acc_remapped = remap_keys(load_data(counts_file)['size_acc'])
    gt_size_acc_remapped = remap_keys(load_data(counts_file)['gt_size_acc'])

    plot_acc_vs_count(size_acc_remapped, 'apartment size (m²)', save_path, 'pred')
    plot_acc_vs_count(gt_size_acc_remapped, 'apartment size (m²)', save_path, 'gt')


def evaluate_observation(prediction_type, config, device):
    with open(config.split_file, "r") as f:
        split = AttrDict(yaml.safe_load(f))
    scene_names = split.test[:config.num_of_scenes] if config.num_of_scenes > 0 else split.test

    if config.use_saved_prob_vol:
        test_set = ProbVolDataset(config.dataset_dir, scene_names, L=config.L, prob_vol_path=config.prob_vol_path, acc_only=False)
    else:
        test_set = GridSeqDataset(config.dataset_dir, scene_names, L=config.L)

    depth_net, semantic_net = None, None
    if not config.use_ground_truth_depth or not config.use_ground_truth_semantic:
        if prediction_type in ["depth", "combined"]:
            depth_net = depth_net_pl.load_from_checkpoint(config.log_dir_depth, d_min=config.d_min, d_max=config.d_max, d_hyp=config.d_hyp, D=config.D).to(device)
        if prediction_type in ["semantic", "combined"]:
            semantic_net = semantic_net_pl.load_from_checkpoint(config.log_dir_semantic, num_classes=config.num_classes).to(device)

    results_type_dir = os.path.join(config.results_dir, prediction_type)
    if config.use_ground_truth_depth or config.use_ground_truth_semantic:
        results_type_dir = os.path.join(results_type_dir, "gt")
    os.makedirs(results_type_dir, exist_ok=True)

    desdfs, semantics, maps, gt_poses, valid_scene_names = load_scene_data(test_set, config.dataset_dir, config.desdf_path)
    evaluate_combined_model(depth_net, semantic_net, desdfs, semantics, test_set, gt_poses, maps, device, results_type_dir, valid_scene_names, config)


def main():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if config.prediction_type == "all":
        for pt in ["depth", "semantic", "combined"]:
            evaluate_observation(pt, config, device)
    else:
        evaluate_observation(config.prediction_type, config, device)


if __name__ == "__main__":
    main()


/home/mascher/anaconda3/envs/yuval_fp/lib/python3.8/site-packages/lightning/fabric/utilities/cloud_io.py:57: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
/home/mascher/anaconda3/envs/yuval_

number of valid scenes for evaluation: 1 out of: 249 --> 0.4016064257028112%
number of valid scenes for evaluation: 2 out of: 249 --> 0.8032128514056224%
number of valid scenes for evaluation: 3 out of: 249 --> 1.2048192771084338%
number of valid scenes for evaluation: 4 out of: 249 --> 1.6064257028112447%
number of valid scenes for evaluation: 5 out of: 249 --> 2.0080321285140563%
number of valid scenes for evaluation: 6 out of: 249 --> 2.4096385542168677%
number of valid scenes for evaluation: 7 out of: 249 --> 2.8112449799196786%
number of valid scenes for evaluation: 8 out of: 249 --> 3.2128514056224895%
number of valid scenes for evaluation: 9 out of: 249 --> 3.614457831325301%
number of valid scenes for evaluation: 10 out of: 249 --> 4.016064257028113%


  6%|▌         | 15/249 [00:00<00:07, 32.91it/s]

number of valid scenes for evaluation: 11 out of: 249 --> 4.417670682730924%
number of valid scenes for evaluation: 12 out of: 249 --> 4.819277108433735%
number of valid scenes for evaluation: 13 out of: 249 --> 5.220883534136546%
number of valid scenes for evaluation: 14 out of: 249 --> 5.622489959839357%
number of valid scenes for evaluation: 15 out of: 249 --> 6.024096385542169%


  9%|▉         | 22/249 [00:00<00:05, 43.86it/s]

...

 96%|█████████▌| 238/249 [00:04<00:00, 51.24it/s]

number of valid scenes for evaluation: 232 out of: 249 --> 93.17269076305222%
number of valid scenes for evaluation: 233 out of: 249 --> 93.57429718875501%
number of valid scenes for evaluation: 234 out of: 249 --> 93.97590361445783%
number of valid scenes for evaluation: 235 out of: 249 --> 94.37751004016064%
number of valid scenes for evaluation: 236 out of: 249 --> 94.77911646586345%
number of valid scenes for evaluation: 237 out of: 249 --> 95.18072289156626%
number of valid scenes for evaluation: 238 out of: 249 --> 95.58232931726907%
number of valid scenes for evaluation: 239 out of: 249 --> 95.98393574297188%
number of valid scenes for evaluation: 240 out of: 249 --> 96.3855421686747%
number of valid scenes for evaluation: 241 out of: 249 --> 96.78714859437751%
number of valid scenes for evaluation: 242 out of: 249 --> 97.18875502008032%


100%|██████████| 249/249 [00:04<00:00, 51.26it/s]


number of valid scenes for evaluation: 243 out of: 249 --> 97.59036144578313%
number of valid scenes for evaluation: 244 out of: 249 --> 97.99196787148594%
number of valid scenes for evaluation: 245 out of: 249 --> 98.39357429718876%
number of valid scenes for evaluation: 246 out of: 249 --> 98.79518072289156%
number of valid scenes for evaluation: 247 out of: 249 --> 99.19678714859438%
number of valid scenes for evaluation: 248 out of: 249 --> 99.59839357429718%
number of valid scenes for evaluation: 249 out of: 249 --> 100.0%

Apartment size bins (m²):
  Bin 0: [29.07, 119.44)
  Bin 1: [119.44, 209.82)
  Bin 2: [209.82, 300.20)
  Bin 3: [300.20, 390.58)
  Bin 4: [390.58, 480.95)
  Bin 5: [480.95, 571.33)
  Bin 6: [571.33, 661.71)
  Bin 7: [661.71, 752.09)
  Bin 8: [752.09, 842.46)
  Bin 9: [842.46, 932.84)
No existing data found at /datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/results/final_results/visualtizations/size_vs_acc/coun

100%|██████████| 6368/6368 [36:04<00:00,  2.94it/s] 
